In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ->  Loading Module and Datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,PowerTransformer,RobustScaler,OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.feature_selection import SelectKBest,f_classif,SelectKBest,mutual_info_regression,f_regression
from sklearn.impute import IterativeImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.model_selection  import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

In [ ]:
train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv',index_col='Id')
test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
submit=pd.DataFrame(test['Id'])

In [ ]:
#------------------------------Deleting columns with null value more than 500 -------------------------------------

null=train.loc[:,train.isnull().sum()>500]
train.drop(null,inplace=True,axis=1)


null=test.loc[:,test.isnull().sum()>500]
test.drop(null,inplace=True,axis=1)



#----------------------------------- Categorical value-----------------------------------------------------------------

encode=LabelEncoder()
for i in train.select_dtypes(include='object').columns:
    train[i]=encode.fit_transform(train[i])
    test[i]=encode.fit_transform(test[i])


    


y=np.log(train['SalePrice'])

test=test.drop('Id',axis=1)


# Modelling and submitting

In [ ]:
model=XGBRegressor(base_score=0.4, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4603, gamma=0.05,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1.7817, monotone_constraints='()',
             n_estimators=2200, n_jobs=4, nthread=-1, num_parallel_tree=1,
             random_state=7, reg_alpha=0.464, reg_lambda=0.8571,
             scale_pos_weight=1, subsample=0.5213,silent = True,tree_method='exact',
             validate_parameters=1, verbosity=0)

In [ ]:
for i in range(1,2):
    pipeline=Pipeline(steps=[('impute',IterativeImputer(max_iter=4)),('model',model)])

    xtrain,xvalid,ytrain,yvalid=train_test_split(train.drop('SalePrice',axis=1),y,test_size=0.2)

    pipeline.fit(xtrain,ytrain)

    yhat=pipeline.predict(xvalid)
    print(np.sqrt(mean_squared_error(yvalid,yhat)),i)

In [ ]:
yhat=pipeline.predict(test)
yhat=np.exp(yhat)

submit['SalePrice']=yhat*0.55+yhat
submit.to_csv('index.csv',index=False)

In [ ]:
submit

# If u found notebook please help by voting it

In [ ]:
train